# Exercise Fourteen: Project Design Starter
In this exercise, you'll be planning out a complex project. You'll draw in some code, but focus on commenting to describe your project structure. The sample document below will guide you through organizing and annotating your project design. The primary components you'll include are:

- Dependencies: What modules will your project need?
- Collection: Where is your data coming from?
- Processing: How will you format and process your data?
- Analysis: What techniques will you use to understand your data?
- Visualization: How will you visualize and explore your data?

# Project Overview
I plan on extracting information from Twitter using #BlerDcon. BlerDcon is a convention — the event’s name signals that it specifically caters to blerds (black nerds), BlerDCon is an inclusive space for all POC, LGBTQ+, female, and disabled fans. Pulling data from the 2021 convention, I seek to gather more information and insight into the topics dicussed around this venue and the sentiments of the participants. 

# Dependencies
Importing modules to extract data from hastags on twitter, compile that information into a digestible list, and then use that info to create visualizations for the data.

In [ ]:
'''# Importing Config to pull credentials for Twitter API
import configparser

#Importing Twitter API
import tweepy

import datetime

import csv

#Importing to help extract data
import requests
from bs4 import BeautifulSoup

#Importing to map out geographical locations of responses
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

#importing to plot findings in visualization
import matplotlib.pyplot as plt

#importing for map visualization
import cartopy.crs as ccrs
from matplotlib.patches import Circle

#importing to assist in compliling 
import re

#Importing for punctuation
from nltk.tokenize import WordPunctTokenizer

#importing to make a wordcloud for visualizations
from wordcloud import WordCloud, STOPWORDS

#To turn work cloud into an image
import numpy as np
from PIL import Image
import random

#Importing to gather sentiments from comments
import nltk.data
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sentiment
from nltk import word_tokenize

# Importing Pandas to handle collected Twitter data 
import pandas as pd

import os'''

In [76]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [79]:
#Pulling credentials file, so Twiiter Developer account can be accessed
CONFIG = configparser.ConfigParser()
CONFIG.read('credentials.ini')

#Tweepy
auth = tweepy.OAuthHandler(CONFIG['DEFAULT']['consumer_key'], CONFIG['DEFAULT']['consumer_secret'])
auth.set_access_token(CONFIG['DEFAULT']['access_token'], CONFIG['DEFAULT']['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True)

'''public_tweets = api.home_timeline()
for tweet in public_tweets:
    #dont need to see everything
    print(tweet.text [0:10])'''

'public_tweets = api.home_timeline()\nfor tweet in public_tweets:\n    #dont need to see everything\n    print(tweet.text [0:10])'

In [80]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

# Collection
Collecting data from recent 2021 convention hastag #BlerDcon. Starting date from pull will begin 01-01-2021 to current time. This can be changed later to look into other year conventions. 

In [ ]:
#pulling info in from specific hashtag
search_words = "#BlerDCon2021"
date_since = "2020-01-01"
tweets = tweepy.Cursor(api.search_tweets,
              q=search_words, lang="en").items(300)

# Processing
After your data has been collected or imported, store it in a format that works for your purposes. This can vary: for Twitter analysis, it might be a Pandas dataframe, while for text, you might build a document term matrix.

In [ ]:
'''#using pandas to sort tweets
tweets_sorted = [[tweet.user.screen_name, tweet.geo, tweet.user.location, tweet.text] for tweet in tweets]

#caluculating coordinates of Tweets -- doesnt look like too many locations (may not use for data set)
tdf = pd.DataFrame(data=tweets_sorted, columns=['user', 'coordinates','location', 'tweet'])
print(tdf)

#may need to look for word count instead'''

In [ ]:
'''BlerDcon_topics = pd.DataFrame(review_dict)
print(BlerDcon_topics)'''

In [ ]:
df = pd.DataFrame(output)
df.to_csv('output.csv')

In [ ]:
#need to organize CSV better then this function will work better

df = pd.read_csv('output.csv')
# Prints how pd will structure data types
print(df.dtypes)

In [ ]:
df = pd.read_csv('output.csv')
df.head()


In [ ]:
#trying to read CSV another way
f = open('output.csv', 'r')
text = f.read()
f.close()

#split_string
word_bag = text.split()
print(word_bag[0:100])

#make lower case
text = text.lower()
word_bag = text.split()
print(word_bag[0:100])

# Analysis
Would like to analyze the CSV file for common key terms and number of times a topic is mentioned. From their try to build out sentiments of users. 

# Visualization
The goal was to create a word cloud with common topics from the hashtag and place it over the logo for the conference. 